In [1]:
from IPython.display import display
import gzip
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error
from math import sqrt
import sys, os
from contextlib import contextmanager
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tcristea\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# 1. Read and parse: goodread_books, goodread_interactions, book_id_map, my_rated_books

In [3]:
def read_authors_dict():
    authors = {}
    with open("inputs/goodreads_book_authors.json", 'r') as f:
        while line := f.readline():
            json_line = json.loads(line)
            authors[json_line["author_id"]] = json_line["name"]
    return authors
    

In [4]:
authors_dict = read_authors_dict()

In [5]:
def get_needed_fields(json_line, authors_dict):
    return {
        "book_id": json_line["book_id"],
        "title": json_line["title"],
        "description": json_line["description"],
        "ratings_count": json_line["ratings_count"],
        "num_pages": json_line["num_pages"],
        "publication_year": json_line["publication_year"],
        "url": json_line["url"],
        # "image_url": json_line["image_url"],
        "average_rating": json_line["average_rating"],
        "authors": " ".join([authors_dict[x["author_id"]] for x in json_line["authors"]]),
        "publisher": json_line["publisher"]
    }

In [6]:
def get_books_df(authors_dict, min_rating_count=1000):
    parsed_books = []
    with gzip.open("inputs/goodreads_books.json.gz", 'r') as f:
        while line := f.readline():
            needed_fields = get_needed_fields(json.loads(line), authors_dict)
            try:
                ratings_count = int(needed_fields["ratings_count"])
            except ValueError:
                continue
            if ratings_count > min_rating_count:
                parsed_books.append(needed_fields)
    books = pd.DataFrame.from_dict(parsed_books)
    books["ratings_count"] = pd.to_numeric(books["ratings_count"])
    books["title"] = books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    books["title"] = books["title"].str.lower()
    books["title"] = books["title"].str.replace("\s+", " ", regex=True)
    books = books[books["title"].str.len() > 0]
    books["book_id"] = pd.to_numeric( books["book_id"])
    return books

In [7]:
books_df = get_books_df(authors_dict, 1000)

In [8]:
books_df

,book_id,title,description,ratings_count,num_pages,publication_year,url,average_rating,authors,publisher
0,6066819,best friends forever,Addie Downs and Valerie Adler were eight when ...,51184,368,2009,https://www.goodreads.com/book/show/6066819-be...,3.49,Jennifer Weiner,Atria Books
1,89375,90 minutes in heaven a true story of death and...,As he is driving home from a minister's confer...,68157,,,https://www.goodreads.com/book/show/89375.90_M...,3.91,Don Piper Cecil Murphey,
2,89376,heaven,What is Heaven really going to be like? What w...,7345,533,,https://www.goodreads.com/book/show/89376.Heaven,4.26,Randy Alcorn,
3,89377,penny from heaven,It's 1953 and 11-year-old Penny dreams of a su...,6949,288,2006,https://www.goodreads.com/book/show/89377.Penn...,3.98,Jennifer L. Holm,Random House Books for Young Readers
4,89378,dog heaven,In Newbery Medalist Cynthia Rylant's classic b...,1331,40,1995,https://www.goodreads.com/book/show/89378.Dog_...,4.43,Cynthia Rylant,Blue Sky Press
...,...,...,...,...,...,...,...,...,...,...
90935,335370,rose madder,A grimmer than Grimm fairy tale for our times-...,1201,420,1995,https://www.goodreads.com/book/show/335370.Ros...,3.67,Stephen King,Viking Adult
90936,57064,hammerfall the gene wars 1,One of the most renowned figures in science fi...,1143,457,2002,https://www.goodreads.com/book/show/57064.Hamm...,3.62,C.J. Cherryh,Eos / Harper Voyager
90937,7715664,sin undone demonica 5,HER TOUCH IS DEADLY\nAs the only female Seminu...,23091,400,2010,https://www.goodreads.com/book/show/7715664-si...,4.35,Larissa Ione,Grand Central Publishing
90938,3106983,persepolis the story of a childhood and the st...,The Story of a Childhood and The Story of a Re...,1966,343,2008,https://www.goodreads.com/book/show/3106983-pe...,4.36,Marjane Satrapi Anjali Singh,Vintage


In [9]:
def get_books_csv_map():
    csv_book_mapping = {}
    first_line = True
    with open("inputs/book_id_map.csv", "r") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if first_line:
                first_line = False
                continue
            csv_id, book_id = line.strip().split(",")
            csv_book_mapping[int(csv_id)] = int(book_id)
    return csv_book_mapping

In [10]:
books_csv_ids_map = get_books_csv_map()

In [11]:
def get_my_rated_books_df(books_df):
    rated_books = []
    first_line = True
    with open("inputs/my_rated_books.csv") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if first_line:
                first_line = False
                continue
            book_id, rating = line.strip().split(",")
            rated_books.append({"book_id": int(book_id), "rating": int(rating)})
    books = pd.DataFrame.from_dict(rated_books, dtype = int)
    books_augmented = books_df.merge(books, how="inner", on="book_id")
    return books_augmented

In [12]:
my_rate_books_df = get_my_rated_books_df(books_df)

In [13]:
def get_filtered_users(my_rate_books_df, same_books_ratio):
    users = {}
    first_line = True
    book_set = set(my_rate_books_df["book_id"])
    with open("inputs/goodreads_interactions.csv") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if first_line:
                first_line = False
                continue
            user_id, csv_id, _, rating, _ = line.strip().split(",")
            book_id = books_csv_ids_map.get(int(csv_id))
            if int(book_id) in book_set:
                if int(user_id) not in users:
                    users[int(user_id)] = 1
                else:
                    users[int(user_id)] +=1
    return set([k for k in users if users[k] > len(my_rate_books_df)/same_books_ratio])

In [14]:
filtered_users = get_filtered_users(my_rate_books_df, 1.5)

In [15]:
def get_interactions_df(filtered_users):
    interactions_lists = []
    first_line = True       
    with open("inputs/goodreads_interactions.csv") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if first_line:
                first_line = False
                continue
            user_id, csv_id, _, rating, _ = line.strip().split(",")
            book_id = books_csv_ids_map.get(int(csv_id))
            if int(user_id) in filtered_users:
                interactions_lists.append({"user_id": int(user_id), "book_id": int(book_id), "rating": int(rating)})

    interactions_df = pd.DataFrame.from_dict(interactions_lists, dtype=int)
    # interactions_df = interactions_df[interactions_df["book_id"].isin(filtered_users)]
    return interactions_df

In [16]:
interactions_df_initial = get_interactions_df(filtered_users)

In [17]:
interactions_df = interactions_df_initial
for i, row in my_rate_books_df[["book_id", "rating"]].iterrows():
    interactions_df = pd.concat([interactions_df, pd.DataFrame([{"user_id": 1, "book_id": row["book_id"], "rating": row["rating"]}])], ignore_index=True)
interactions_df = interactions_df.sort_values(by="user_id").reset_index(drop=True)   

In [18]:
# 2. Preprocessing of DataFrames

In [19]:
def preprocess_books_df(books_df):
    def removeNonAscii(s):
        return "".join(i for i in s if  ord(i)<128)
    def make_lower_case(text):
        return text.lower()
    def remove_stop_words(text):
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
        return text
    def remove_punctuation(text):
        tokenizer = RegexpTokenizer(r'\w+')
        text = tokenizer.tokenize(text)
        text = " ".join(text)
        return text
    def full_normalize(text):
        text = re.sub("[^a-zA-Z0-9 ]", "", text)
        text = re.sub("\s+", " ", text)
        text = removeNonAscii(text)
        text = make_lower_case(text)
        text = remove_stop_words(text)
        return remove_punctuation(text)
        
    aux_df = books_df.copy()
    aux_df["title"] = aux_df["title"].apply(full_normalize)
    aux_df = aux_df[aux_df["title"].str.len() > 0]
    aux_df["description"] = aux_df["description"].apply(full_normalize)
    aux_df = aux_df[aux_df["description"].str.len() > 0]
    aux_df["authors"] = aux_df["authors"].apply(full_normalize)
    aux_df = aux_df[aux_df["authors"].str.len() > 0]
    aux_df["publisher"] = aux_df["publisher"].apply(full_normalize)
    aux_df = aux_df[aux_df["publisher"].str.len() > 0]
    # TODO migh need/not need this for average_rating
    # aux_df["average_rating"] = aux_df["average_rating"].apply(full_normalize)
    # aux_df = aux_df[aux_df["average_rating"].str.len() > 0]
    
    
    return aux_df

In [20]:
books_df_initial = books_df
books_df = preprocess_books_df(books_df)

In [21]:
books_df

,book_id,title,description,ratings_count,num_pages,publication_year,url,average_rating,authors,publisher
0,6066819,best friends forever,addie downs valerie adler eight first met deci...,51184,368,2009,https://www.goodreads.com/book/show/6066819-be...,3.49,jennifer weiner,atria books
3,89377,penny heaven,1953 11yearold penny dreams summer butter peca...,6949,288,2006,https://www.goodreads.com/book/show/89377.Penn...,3.98,jennifer l holm,random house books young readers
4,89378,dog heaven,newbery medalist cynthia rylants classic bests...,1331,40,1995,https://www.goodreads.com/book/show/89378.Dog_...,4.43,cynthia rylant,blue sky press
5,25317381,smoke,amazon best book yeara goodreads best book mon...,1537,340,2015,https://www.goodreads.com/book/show/25317381-s...,3.69,catherine mckenzie,lake union
7,22642971,body electric,future world peaceella shepherd dedicated life...,1525,351,2014,https://www.goodreads.com/book/show/22642971-t...,3.71,beth revis,scripturient books
...,...,...,...,...,...,...,...,...,...,...
90934,7657484,superman earth one volume 1,forget everything know man steel brace stagger...,13221,134,2010,https://www.goodreads.com/book/show/7657484-su...,3.90,j michael straczynski shane davis sandra hope ...,dc comics
90935,335370,rose madder,grimmer grimm fairy tale timesfrom master maca...,1201,420,1995,https://www.goodreads.com/book/show/335370.Ros...,3.67,stephen king,viking adult
90936,57064,hammerfall gene wars 1,one renowned figures science fiction cj cherry...,1143,457,2002,https://www.goodreads.com/book/show/57064.Hamm...,3.62,cj cherryh,eos harper voyager
90937,7715664,sin undone demonica 5,touch deadlyas female seminus demon ever born ...,23091,400,2010,https://www.goodreads.com/book/show/7715664-si...,4.35,larissa ione,grand central publishing


In [22]:
# 3. Recommender systems have a problem known as user cold-start, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.
#     For this reason, we are keeping in the dataset only users with at least 5 interactions.

In [23]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'book_id']).size().groupby('user_id').size()

In [24]:
print('# users: %d' % len(users_interactions_count_df))

# users: 625


In [25]:
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]

In [26]:
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users with at least 5 interactions: 625


In [27]:
print('# of interactions: %d' % len(interactions_df))

# of interactions: 2878532


In [28]:
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')

In [29]:
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions from users with at least 5 interactions: 2878532


In [30]:
# Our User Interactions dataset is complete and all users have at least 5 interactions.

In [31]:
# 4. We aggregate all the interactions a user had with a book (review) by a weighted sum of interaction-type strenght.
    # The weight in our case is the rating (0-5)
    # We then apply a log transformation to smooth the distribution
# TODO: We can also skip this step, as all our interactions are unique (user_id+book_id grouping). This step will only
    # apply the log transformation in our case
    # to skip this step just use
    # interactions_full_df = interactions_from_selected_users_df

In [32]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [33]:
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'book_id'])['rating'].sum() \
                    .apply(smooth_user_preference).reset_index()

In [34]:
print('# of unique user/item interactions: %d' % len(interactions_full_df))

# of unique user/item interactions: 2878532


In [35]:
interactions_full_df.head()

,user_id,book_id,rating
0,1,2373,2.584963
1,1,142540,2.584963
2,1,227729,2.000000
3,1,395851,2.321928
4,1,395875,2.584963


In [36]:
# 5. Create our TF-IDF vector
    # Ignoring stopwords (words with no semantics) from English
    # Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus
# TODO:
    # - We can adjust the 5000 vector size and check for better/worse results (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [37]:
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords.words('english'))

In [72]:
vectorizer

TfidfVectorizer(max_df=0.5, max_features=5000, min_df=0.003, ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [38]:
book_ids = books_df['book_id'].tolist()

In [39]:
# In our TF-IDF we use: title, description, ratings_count, num_pages, publication_year, average_rating, authors and publisher
# TODO:
    # - We can only use title, description as well

In [85]:
tfidf_matrix = vectorizer.fit_transform(books_df['title'] + " " + books_df['description'] + " " + books_df['ratings_count'].astype(str)+ " " + \
                                       books_df['num_pages'].astype(str) + " " +  books_df['publication_year'].astype(str) + " " + \
                                       books_df['average_rating'].astype(str) + " " + books_df['authors'])

In [41]:
tfidf_matrix

<66087x4656 sparse matrix of type '<class 'numpy.float64'>'
	with 3779569 stored elements in Compressed Sparse Row format>

In [42]:
# 6.0. To create user profiles, we take all the books the user interacted with and average them. The average is weighted
    # by the rating. The books the user has rated the highest will have a higher strangth in the final user profile.

In [86]:
def get_book_profile(book_id):
    idx = book_ids.index(book_id)
    book_profile = tfidf_matrix[idx:idx+1]
    return book_profile

In [87]:
def get_book_profiles(book_ids):
    book_profiles_list = [get_book_profile(x) for x in book_ids]
    book_profiles = scipy.sparse.vstack(book_profiles_list)
    return book_profiles

In [88]:
def build_users_profile(user_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[user_id]
    user_book_profiles = get_book_profiles(interactions_person_df['book_id'])
    
    user_book_strengths = np.array(interactions_person_df['rating']).reshape(-1,1)
    # print(user_book_strengths)
    # print(user_id)
        
    #Weighted average of item profiles by the interactions strength
    aux_copy = user_book_strengths.copy()
    aux_copy[aux_copy == 0] = 1
    user_book_strengths_weighted_avg = np.sum(user_book_profiles.multiply(aux_copy), axis=0) / np.sum(user_book_strengths)
    # print(user_book_strengths_weighted_avg)
    # print(np.asarray(user_book_strengths_weighted_avg))
    user_profile_norm = sklearn.preprocessing.normalize(np.asarray(user_book_strengths_weighted_avg))
    return user_profile_norm

In [89]:
def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['book_id'] \
                                                   .isin(books_df['book_id'])].set_index('user_id')
    user_profiles = {}
    for user_id in interactions_indexed_df.index.unique():
        user_profiles[user_id] = build_users_profile(user_id, interactions_indexed_df)
    return user_profiles

In [90]:
# 6.1. Split user interactions in 2 datasets: train (80%) and test (20%).

In [91]:
# interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
#                                    stratify=interactions_full_df['user_id'], 
#                                    test_size=0.20,
#                                    random_state=42)
interactions_train_df = interactions_full_df

In [92]:
print('# interactions on Train set: %d' % len(interactions_train_df))
# print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 2878532


In [93]:
# Warning: Running this takes considerable time: X mins

In [94]:
user_profiles = build_users_profiles()

In [95]:
len(user_profiles)

625

In [96]:
# 7. Let's take a look at the user profile for user_id = 1. This is a vector of 5000 tokens together with
    # how relevant is that token for user_id=1 (tokens are both unigrams and bigrams)
    # We can sort the tokens by their relevance

In [97]:
myprofile = user_profiles[1]

In [98]:
myprofile

array([[0.        , 0.03305183, 0.02433519, ..., 0.        , 0.        ,
        0.        ]])

In [99]:
print(myprofile.shape)

(1, 4466)


In [100]:
tfidf_feature_names = vectorizer.get_feature_names_out()

In [101]:
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[1].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,throne,0.317354
1,glass,0.292706
2,sarah,0.223369
3,assassin,0.199357
4,shatter,0.191613
5,vampire,0.180695
6,unspeakable,0.145393
7,court,0.143830
8,diaries,0.120552
9,series,0.108410


In [103]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, books_df=None):
        self.book_ids = book_ids
        self.books_df = books_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_books_to_user_profile(self, user_id, topn=1000):
        #Computes the cosine similarity between the user profile and all book profiles
        cosine_similarities = cosine_similarity(user_profiles[user_id], tfidf_matrix)
        #Gets the top similar books
        similar_indices = cosine_similarities.argsort().flatten()
        #Sort the similar books by similarity
        similar_items = sorted([(book_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_books(self, user_id, books_to_ignore=[], topn=10, verbose=False):
        similar_books = self._get_similar_books_to_user_profile(user_id)
        #Ignores books the user has already rated
        similar_books_filtered = list(filter(lambda x: x[0] not in books_to_ignore, similar_books))
        
        recommendations_df = pd.DataFrame(similar_books_filtered, columns=['book_id', 'recStrength'])

        if verbose:
            if self.books_df is None:
                raise Exception('"books_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.books_df, how = 'left', 
                                                          left_on = 'book_id', 
                                                          right_on = 'book_id')[['recStrength', 'book_id', 'title', 'description', 'ratings_count', \
                                                                                 'url', 'average_rating', 'authors', 'publisher', \
                                                                                'publication_year', 'num_pages']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(books_df)

In [104]:
# 8. Before evaluating the model, lets see what books are recommended for user_id = 1

In [105]:
def get_books_interacted(user_id, interactions_df):
    interacted_books = interactions_df.loc[user_id]['book_id']
    return set(interacted_books if type(interacted_books) == pd.Series else [interacted_books])

In [106]:
# user_1_recommendations = content_based_recommender_model.recommend_books(1, get_books_interacted(1, interactions_df.set_index('user_id')), 20, True)
user_1_recommendations = content_based_recommender_model.recommend_books(1, {}, 20, True)

In [107]:
# 9. TODO: Evaluate Model

In [108]:
# user_1_recommendations[user_1_recommendations["book_id"] == user_1_recommendations["recStrength"].min()]
# user_1_recommendations[user_1_recommendations["ratings_count"] > 10000].head(30)
user_1_recommendations[user_1_recommendations["book_id"] == 2373].empty
user_1_recommendations.head(10)

,recStrength,book_id,title,description,ratings_count,url,average_rating,authors,publisher,publication_year,num_pages
0,0.457660,13565676,assassin empire throne glass 05,celaena sardothien assassin everything place c...,18049,https://www.goodreads.com/book/show/13565676-t...,4.42,sarah j maas,bloomsbury usa childrens,2012,94
1,0.443082,33590260,untitled throne glass 7,final installment throne glass series,1190,https://www.goodreads.com/book/show/33590260-u...,4.63,sarah j maas,bloomsbury usa childrens books,2018,702
2,0.438609,28260587,empire storms throne glass 5,kingdoms collidethe long path throne begun ael...,70571,https://www.goodreads.com/book/show/28260587-e...,4.58,sarah j maas,bloomsbury childrens books,2016,693
3,0.427192,13415554,assassin pirate lord throne glass 01,throne glass novellaon remote island tropical ...,21641,https://www.goodreads.com/book/show/13415554-t...,4.23,sarah j maas,bloomsbury childrens,2012,70
4,0.426003,18333581,assassin healer throne glass 02,meet assassin beautiful defiant destined great...,11992,https://www.goodreads.com/book/show/18333581-t...,4.21,sarah j maas,bloomsbury publishing,2013,40
5,0.423650,18006496,queen shadows throne glass 4,queen returnedeveryone celaena sardothien love...,106182,https://www.goodreads.com/book/show/18006496-q...,4.60,sarah j maas,bloomsbury usa childrens,2015,648
6,0.419664,17167166,crown midnight throne glass 2,line never crossed breachedit puts entire cast...,169307,https://www.goodreads.com/book/show/17167166-c...,4.49,sarah j maas,bloomsbury usa childrens,2013,418
7,0.419450,23599075,throne glass throne glass 1,nothing coincidence everything purpose meant c...,3236,https://www.goodreads.com/book/show/23599075-t...,4.23,sarah j maas,bloomsbury usa childrens,2013,404
8,0.409438,7896527,throne glass throne glass 1,alternate cover edition found serving year har...,295609,https://www.goodreads.com/book/show/7896527-th...,4.23,sarah j maas,bloomsbury usa childrens,2012,404
9,0.403699,13574861,throne glass throne glass 1,alternate cover edition asin serving year hard...,9687,https://www.goodreads.com/book/show/13574861-t...,4.23,sarah j maas,bloomsbury usa childrens,2012,432


In [121]:
aux = user_1_recommendations[["recStrength", "book_id"]].copy()
aux["recStrength"] = aux["recStrength"] * 10

In [122]:
def evaluate_predictions(user_recommendations, my_rate_books_df, metric="Content-Based"):
    predictions = []
    target = []
    predictions_to_ret = {}
    for i, row in my_rate_books_df.iterrows():
        target.append(row["rating"])
        # predictions.append(svd.predict(1, row["book_id"]).est)
        predictions.append(float(user_recommendations[user_recommendations["book_id"] == row["book_id"]]["recStrength"].iloc[0]) if \
                          not user_recommendations[user_recommendations["book_id"] == row["book_id"]].empty else 4.02)
        predictions_to_ret[row["book_id"]] = (float(user_recommendations[user_recommendations["book_id"] == row["book_id"]]["recStrength"].iloc[0]) if \
                          not user_recommendations[user_recommendations["book_id"] == row["book_id"]].empty else 4.02, row["rating"])
        # print(row)
    prediction= pd.DataFrame(predictions).T
    target_prediction = pd.DataFrame(target).T

    # 1. Compute RMSE
    RS = np.sqrt(np.mean((prediction-target_prediction)**2))
    print("Valoarea RMSE pentru predictia de tip {} este: {}".format(metric, RS))

    # 2. Compute MAE
    mae = mean_absolute_error(target_prediction, prediction)
    print("Valoarea MAE pentru predictia de tip {} este: {}".format(metric, mae))

    TPs = 0
    FNs = 0
    # The number of relevant items are the items with actual rating greater or equal to 3.5.
    threshold = 3.5
     # Find the relevant items using the threshold
    relevant_items = []
    for i in range(0, target_prediction.shape[0]):
        for j in range(0, target_prediction.shape[1]):
            if target_prediction.values[i, j] > threshold:
                relevant_items.append((i, j))

    # Compute K
    k = len(relevant_items)

    # Recommended items @ k
    recommended_items_at_k = []
    for i in range(0, target_prediction.shape[0]):
        for j in range(0, target_prediction.shape[1]):
            if prediction.values[i, j] > threshold:
                recommended_items_at_k.append((i, j))

     # Recommended and Relevant items @ k (Intersection)
    recomm_and_relevant_items_at_k = list(set(relevant_items) & set(recommended_items_at_k))

    # Compute Precision @ K
    precision_at_k = len(recomm_and_relevant_items_at_k) / len(recommended_items_at_k)

    print("Precision at k={}, for prediction using metric {}, is: {}".format(k, metric, precision_at_k))

    # Compute Recall @ K
    recall_at_k = len(recomm_and_relevant_items_at_k) / len(relevant_items)

    print("Recall at k={} for prediction using metric {}, is: {}".format(k, metric, recall_at_k))

    # Compute F1 score @ K
    f1_score_at_k = 2 * precision_at_k * recall_at_k / (precision_at_k + recall_at_k)

    print("F1 Score at k={}, for prediction using metric {}, is: {}".format(k, metric, f1_score_at_k))
    return predictions_to_ret
    

In [123]:
content_based_predictions = evaluate_predictions(aux, my_rate_books_df)

Valoarea RMSE pentru predictia de tip Content-Based este: 1.451388584164388
Valoarea MAE pentru predictia de tip Content-Based este: 1.1930588591264004
Precision at k=22, for prediction using metric Content-Based, is: 0.8666666666666667
Recall at k=22 for prediction using metric Content-Based, is: 0.5909090909090909
F1 Score at k=22, for prediction using metric Content-Based, is: 0.7027027027027029


In [124]:
content_based_predictions

{13104080: (2.6969770317900092, 5),
 395875: (2.8247334348241755, 5),
 1169808: (3.692906927282708, 5),
 10429045: (2.3976474680371425, 4),
 13565676: (4.576600828467035, 4),
 23766634: (3.196370620052771, 3),
 13419891: (3.614129202702699, 5),
 142540: (1.9270116545053873, 5),
 395922: (3.4699940919769547, 5),
 20613470: (3.9117361648899522, 5),
 29008738: (2.4728692617307715, 5),
 18006496: (4.236503104889732, 4),
 13415554: (4.271920132072667, 5),
 17331518: (3.0052962306506856, 5),
 17167166: (4.196642057447526, 4),
 28260587: (4.386086052082112, 5),
 31450852: (4.02, 5),
 395851: (3.7805349790448624, 4),
 13188676: (3.1469505320354556, 5),
 18333581: (4.2600332471557545, 4),
 2373: (4.02, 5),
 16096824: (2.8351461703389154, 5),
 17927395: (3.7198854582095837, 3),
 7896527: (4.0943754959021055, 4),
 227729: (4.02, 3)}

In [113]:
# Hybrid Model

In [114]:
svd_predictions = {13104080: (5, 5),
 395875: (4.623304667079186, 5),
 1169808: (4.700428011513711, 5),
 10429045: (4.69773755557114, 4),
 13565676: (4.476495798083327, 4),
 23766634: (3.735443965722295, 3),
 13419891: (4.714168739575217, 5),
 142540: (3.184105938606115, 5),
 395922: (4.704848494599006, 5),
 20613470: (4.203287233539408, 5),
 29008738: (3.4050771012860905, 5),
 18006496: (4.378874537661045, 4),
 13415554: (4.598957728959821, 5),
 17331518: (4.149879583368218, 5),
 17167166: (4.379493769462175, 4),
 28260587: (3.9556038446562205, 5),
 31450852: (3.6987781565071796, 5),
 395851: (4.663370677685508, 4),
 13188676: (4.914707767958816, 5),
 18333581: (4.452522676938357, 4),
 2373: (3.7060847380882143, 5),
 16096824: (4.787223584601609, 5),
 17927395: (4.127090922908268, 3),
 7896527: (4.431188944194735, 4),
 227729: (3.4173784513395544, 3)}

In [115]:
def evaluate_hybrid_model_predictions(cotent_based_predictions, item_based_predictions, cb_weight, ib_weight, metric="Hybrid"):
    predictions = []
    target = []
    for key, value in cotent_based_predictions.items():
        target.append(value[1])
        pred_value = (value[0] * cb_weight + item_based_predictions[key][0] * ib_weight) / (cb_weight + ib_weight)
        predictions.append(pred_value)
        # print(row)
    prediction= pd.DataFrame(predictions).T
    target_prediction = pd.DataFrame(target).T

    # 1. Compute RMSE
    RS = np.sqrt(np.mean((prediction-target_prediction)**2))
    print("Valoarea RMSE pentru predictia de tip {} este: {}".format(metric, RS))

    # 2. Compute MAE
    mae = mean_absolute_error(target_prediction, prediction)
    print("Valoarea MAE pentru predictia de tip {} este: {}".format(metric, mae))
    

    TPs = 0
    FNs = 0
    # The number of relevant items are the items with actual rating greater or equal to 3.5.
    threshold = 3.5
     # Find the relevant items using the threshold
    relevant_items = []
    for i in range(0, target_prediction.shape[0]):
        for j in range(0, target_prediction.shape[1]):
            if target_prediction.values[i, j] > threshold:
                relevant_items.append((i, j))

    # Compute K
    k = len(relevant_items)

    # Recommended items @ k
    recommended_items_at_k = []
    for i in range(0, target_prediction.shape[0]):
        for j in range(0, target_prediction.shape[1]):
            if prediction.values[i, j] > threshold:
                recommended_items_at_k.append((i, j))

     # Recommended and Relevant items @ k (Intersection)
    recomm_and_relevant_items_at_k = list(set(relevant_items) & set(recommended_items_at_k))

    # Compute Precision @ K
    precision_at_k = len(recomm_and_relevant_items_at_k) / len(recommended_items_at_k)

    print("Precision at k={}, for prediction using metric {}, is: {}".format(k, metric, precision_at_k))

    # Compute Recall @ K
    recall_at_k = len(recomm_and_relevant_items_at_k) / len(relevant_items)

    print("Recall at k={} for prediction using metric {}, is: {}".format(k, metric, recall_at_k))

    # Compute F1 score @ K
    f1_score_at_k = 2 * precision_at_k * recall_at_k / (precision_at_k + recall_at_k)

    print("F1 Score at k={}, for prediction using metric {}, is: {}".format(k, metric, f1_score_at_k))
    

In [116]:
evaluate_hybrid_model_predictions(content_based_predictions, svd_predictions, 1, 4)

Valoarea RMSE pentru predictia de tip Hybrid este: 0.8601555606034893
Valoarea MAE pentru predictia de tip Hybrid este: 0.734683225673875
Precision at k=22, for prediction using metric Hybrid, is: 0.8695652173913043
Recall at k=22 for prediction using metric Hybrid, is: 0.9090909090909091
F1 Score at k=22, for prediction using metric Hybrid, is: 0.888888888888889
